<a href="https://colab.research.google.com/github/SIDED00R/machinelearning/blob/main/deep4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from torch import nn, optim, from_numpy
import numpy as np
from google.colab import drive
drive.mount('/content/gdrive')
xy=np.loadtxt('/content/gdrive/MyDrive/Colab Notebooks/diabetes.csv.gz', delimiter=',', dtype=np.float32) 
x_data=from_numpy(xy[:, 0:-1])
y_data=from_numpy(xy[:, [-1]])
print(f'X\'s shape: {x_data.shape} | Y\'s shape: {y_data.shape}')

Mounted at /content/gdrive
X's shape: torch.Size([759, 8]) | Y's shape: torch.Size([759, 1])


In [2]:
from torch import nn, optim, from_numpy
import numpy as np
from google.colab import drive
from torch.utils.data import Dataset, DataLoader
drive.mount('/content/gdrive')
class DiabetesDataset(Dataset):
  def __init__(self):
    xy=np.loadtxt('/content/gdrive/MyDrive/Colab Notebooks/diabetes.csv.gz', delimiter=',', dtype=np.float32) 
    self.x_data=from_numpy(xy[:, 0:-1])
    self.y_data=from_numpy(xy[:, [-1]])
    self.len=xy.shape[0]

  def __getitem__(self,index):
      return self.x_data[index], self.y_data[index]

  def __len__(self):
      return self.len

dataset_=DiabetesDataset()
train_loader=DataLoader(dataset= dataset_, batch_size=32, shuffle=True, num_workers=2)

class Model(nn.Module):
  def __init__(self):
    super(Model,self).__init__()
    self.l1=nn.Linear(8,6)
    self.l2=nn.Linear(6,4)
    self.l3=nn.Linear(4,1)
    self.sigmoid=nn.Sigmoid()
    self.relu=nn.ReLU()

  def forward(self,x):
    out1=self.relu(self.l1(x))
    out2=self.relu(self.l2(out1))
    y_pred=self.sigmoid(self.l3(out2))
    return y_pred

model=Model()
criterion=nn.BCELoss(reduction='mean')
optimizer=optim.SGD(model.parameters(), lr=0.1)
for epoch in range(100):
  for i, data in enumerate(train_loader,0):
    inputs,labels=data
    y_pred=model(inputs)
    loss=criterion(y_pred,labels)
    print(f'Epoch{epoch+1} | Batch: {i+1} Loss: {loss.item(): .4f}')
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()



Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
Epoch1 | Batch: 1 Loss:  0.7114
Epoch1 | Batch: 2 Loss:  0.7166
Epoch1 | Batch: 3 Loss:  0.7251
Epoch1 | Batch: 4 Loss:  0.6984
Epoch1 | Batch: 5 Loss:  0.7136
Epoch1 | Batch: 6 Loss:  0.6986
Epoch1 | Batch: 7 Loss:  0.6995
Epoch1 | Batch: 8 Loss:  0.6937
Epoch1 | Batch: 9 Loss:  0.6958
Epoch1 | Batch: 10 Loss:  0.6963
Epoch1 | Batch: 11 Loss:  0.6923
Epoch1 | Batch: 12 Loss:  0.6891
Epoch1 | Batch: 13 Loss:  0.6824
Epoch1 | Batch: 14 Loss:  0.6886
Epoch1 | Batch: 15 Loss:  0.6819
Epoch1 | Batch: 16 Loss:  0.6701
Epoch1 | Batch: 17 Loss:  0.6805
Epoch1 | Batch: 18 Loss:  0.6837
Epoch1 | Batch: 19 Loss:  0.6707
Epoch1 | Batch: 20 Loss:  0.6735
Epoch1 | Batch: 21 Loss:  0.6711
Epoch1 | Batch: 22 Loss:  0.6701
Epoch1 | Batch: 23 Loss:  0.6629
Epoch1 | Batch: 24 Loss:  0.6678
Epoch2 | Batch: 1 Loss:  0.6466
Epoch2 | Batch: 2 Loss:  0.6924
Epoch2 | Batch: 3 Loss

In [3]:
# https://github.com/pytorch/examples/blob/master/mnist/main.py
from __future__ import print_function
from torch import nn, optim, cuda
from torch.utils import data
from torchvision import datasets, transforms
import torch.nn.functional as F
import time

# Training settings
batch_size = 64
device = 'cuda' if cuda.is_available() else 'cpu'
print(f'Training MNIST Model on {device}\n{"=" * 44}')

# MNIST Dataset
train_dataset = datasets.MNIST(root='./mnist_data/',
                               train=True,
                               transform=transforms.ToTensor(),
                               download=True)

test_dataset = datasets.MNIST(root='./mnist_data/',
                              train=False,
                              transform=transforms.ToTensor())

# Data Loader (Input Pipeline)
train_loader = data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)


class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.mp = nn.MaxPool2d(2)
        self.fc = nn.Linear(320, 10)

    def forward(self, x):
        in_size = x.size(0)
        x = F.relu(self.mp(self.conv1(x)))
        x = F.relu(self.mp(self.conv2(x)))
        x = x.view(in_size, -1)  # flatten the tensor
        x = self.fc(x)
        return F.log_softmax(x)


model = Net()
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)


def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} | Batch Status: {}/{} ({:.0f}%) | Loss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))


def test():
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        data, target = data.to(device), target.to(device)
        output = model(data)
        # sum up batch loss
        test_loss += criterion(output, target).item()
        # get the index of the max
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    test_loss /= len(test_loader.dataset)
    print(f'===========================\nTest set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)} '
          f'({100. * correct / len(test_loader.dataset):.0f}%)')


if __name__ == '__main__':
    since = time.time()
    for epoch in range(1, 10):
        epoch_start = time.time()
        train(epoch)
        m, s = divmod(time.time() - epoch_start, 60)
        print(f'Training time: {m:.0f}m {s:.0f}s')
        test()
        m, s = divmod(time.time() - epoch_start, 60)
        print(f'Testing time: {m:.0f}m {s:.0f}s')

    m, s = divmod(time.time() - since, 60)
    print(f'Total Time: {m:.0f}m {s:.0f}s\nModel was trained on {device}!')



Training MNIST Model on cpu


  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./mnist_data/MNIST/raw/train-images-idx3-ubyte.gz to ./mnist_data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./mnist_data/MNIST/raw/train-labels-idx1-ubyte.gz to ./mnist_data/MNIST/raw



...

Extracting ./mnist_data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./mnist_data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ./mnist_data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./mnist_data/MNIST/raw

Train Epoch: 1 | Batch Status: 0/60000 (0%) | Loss: 2.299730


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:49: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


Train Epoch: 1 | Batch Status: 640/60000 (1%) | Loss: 2.293741
Train Epoch: 1 | Batch Status: 1280/60000 (2%) | Loss: 2.270647
Train Epoch: 1 | Batch Status: 1920/60000 (3%) | Loss: 2.227532
Train Epoch: 1 | Batch Status: 2560/60000 (4%) | Loss: 2.209363
Train Epoch: 1 | Batch Status: 3200/60000 (5%) | Loss: 2.174132
Train Epoch: 1 | Batch Status: 3840/60000 (6%) | Loss: 2.094544
Train Epoch: 1 | Batch Status: 4480/60000 (7%) | Loss: 1.973746
Train Epoch: 1 | Batch Status: 5120/60000 (9%) | Loss: 1.858489
Train Epoch: 1 | Batch Status: 5760/60000 (10%) | Loss: 1.596246
Train Epoch: 1 | Batch Status: 6400/60000 (11%) | Loss: 1.344402
Train Epoch: 1 | Batch Status: 7040/60000 (12%) | Loss: 1.086075
Train Epoch: 1 | Batch Status: 7680/60000 (13%) | Loss: 0.929347
Train Epoch: 1 | Batch Status: 8320/60000 (14%) | Loss: 0.717807
Train Epoch: 1 | Batch Status: 8960/60000 (15%) | Loss: 0.752362
Train Epoch: 1 | Batch Status: 9600/60000 (16%) | Loss: 0.572052
Train Epoch: 1 | Batch Status: 102